#### Why Snowpark?
The purpose of Snowpark lies in its ability to provide more flexibility, scalability, and integration for data processing and orchestration tasks.|

Orchestrating Jobs and Pipelines: We can automate Snowpark-based pipelines (UDFs, views, and other transformations) in a more flexible way. Snowpark code can be versioned and tested, making it easier to maintain and extend in your pipeline.

## TO NEHAA:
### GO TO CELL 1
### GET TO WORK NOW

#### Note:
This notebook is to validate, clean and update data in tables of higher priority in terms of updating speed.

Data tables that need a faster updating speed will be processed here.

#### Desired Flow
1. Grab data from raw tables
2. Check for data, validate, check and update cleaned data onto the cleaned tables
3. Join tables via foreign keys and make hybrid master table
4. Create Schema Views

#### 1. Loading Raw Data In

Since we already push our data onto Snowflake, we can call for them in this notebook to run in Snowpark. This will be the first step to the data flow overview for establishing the CI/CD deployment & finalizing the ELT pipepline.

##### Tables in this notebook
- Sales.CustomerTransactions 
- Sales.CustomerCategories 
- Application.People 
- Purchasing.SupplierTransactions 
- Purchasing.PurchaseOrderLines 
- Warehouse.ColdRoomTemperatures 
- Warehouse.VehicleTemperatures
- Sales.Orders 
- Purchasing.PurchaseOrders
- Warehouse.StockItemTransactions
- Warehouse.StockItems
- Warehouse.StockItemHoldings
- Sales.Customers
- Sales.Invoices
- Sales.InvoiceLines
- Sales.Orderlines

In [ ]:
# RAW TABLES
# ---------------------------------------------------------------------------------------------
import time
from snowflake.snowpark import Session

# Define all tables organized by categories or schemas (CALL FOR RAW TABLES)
TABLE_DICT = {
    "application": {
        "schema": "KN_LOGISTICS.SNOWSQL", 
        "tables": [
            "APPLICATION_PEOPLE_RAW"
        ]
    },
    "purchasing": {
        "schema": "KN_LOGISTICS.SNOWSQL",
        "tables": [
            "PURCHASING_PURCHASEORDERLINES_RAW",
            "PURCHASING_PURCHASEORDERS_RAW"
        ]
    },
    "sales": {
        "schema": "KN_LOGISTICS.SNOWSQL",
        "tables": [
            "SALES_CUSTOMERCATEGORIES_RAW",
            "SALES_CUSTOMERS_RAW",
            "SALES_CUSTOMERTRANSACTIONS_RAW",
            "SALES_INVOICELINES_RAW",
            "SALES_INVOICES_RAW",
            "SALES_ORDERLINES_RAW",
            "SALES_ORDERS_RAW"
        ]
    },
    "warehouse": {
        "schema": "KN_LOGISTICS.SNOWSQL",
        "tables": [
            "WAREHOUSE_COLDROOMTEMPERATURES_RAW",
            "WAREHOUSE_STOCKITEMHOLDINGS_RAW",
            "WAREHOUSE_STOCKITEMS_RAW",
            "WAREHOUSE_STOCKITEMTRANSACTIONS_RAW",
            "WAREHOUSE_VEHICLETEMPERATURES_RAW"
        ]
    }
}

def load_raw_table(session, tname=None, schema=None):
    # Adjusted for direct use (no S3 staging assumed in your case)
    session.use_schema(schema)
    print(f"Loading table: {tname}")
    # If additional logic for transformations/loading is needed, add it here
    df = session.table(tname)
    df.show()  # Example action to verify table content

def load_all_tables(session):
    for category, data in TABLE_DICT.items():
        schema = data['schema']
        tables = data['tables']
        for tname in tables:
            load_raw_table(session, tname=tname, schema=schema)

def validate_tables(session):
    for category, data in TABLE_DICT.items():
        schema = data['schema']
        tables = data['tables']
        for tname in tables:
            session.use_schema(schema)
            print(f"Validating table: {tname}")
            print(f"Columns: {session.table(tname).columns}")

In [ ]:
# Add the utils package to our path and import the snowpark_utils function
import os, sys
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

In [ ]:
#from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
load_all_tables(session)

In [ ]:
validate_tables(session)

#### 2. Check for data, validate, check and update cleaned data onto the cleaned tables
After pulling in the raw data, we can check and validate these raw data to make sure they are of a certain format eligible to be pushed over to the cleaned tables. 

If not, we will clean the tables accordingly then update them over to the cleaned tables.

Some of the validation we can do is:
- Check for null values
- Check for duplicates in PK and UQ
- Check for invalid datetypes & text formats

After cleaning the data, we need to validate ONE MORE TIME to make sure the data has not been imported into the cleaned tables before. If validation succeeds, we update the current records over to the cleaned tables.
- Check if the data records exist in the cleaned tables (checkj if they are identitcal)

In [ ]:
TABLE_CONFIG = {
    "APPLICATION_PEOPLE_RAW": {
        "clean_table": "APPLICATION_PEOPLE",
        "primary_keys": ["PERSON_ID"],
        "unique_keys": [],
        "date_format": "%Y-%m-%d"
    },
    "PURCHASING_SUPPLIERTRANSACTIONS_RAW": {
        "clean_table": "PURCHASING_SUPPLIERTRANSACTIONS",
        "primary_keys": ["SUPPLIERTRANSACTIONID"],
        "unique_keys": [],
        "date_format": "%Y-%m-%d"
    },
    "PURCHASING_PURCHASEORDERS_RAW": {
        "clean_table": "PURCHASING_PURCHASEORDERS",
        "primary_keys": ["PURCHASEORDERID"],
        "unique_keys": [],
        "date_format": "%Y-%m-%d"
    },
    # Add all tables here...
}


In [ ]:
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col

def clean_table(session: Session, raw_table: str, config: dict):
    """
    Cleans a raw table based on the provided configuration.

    Args:
        session (Session): Snowflake Snowpark session.
        raw_table (str): Fully qualified name of the raw table (schema.table).
        config (dict): Configuration for the table.

    Returns:
        DataFrame: Cleaned DataFrame.
    """
    print(f"Cleaning table: {raw_table}")
    df = session.table(raw_table)

    # Replace 'NULL' strings with actual NULL values
    cleaned_df = df.replace({"NULL": None})

    # Handle date columns
    if "date_format" in config:
        for column in df.columns:
            cleaned_df = cleaned_df.with_column(column, col(column).cast("date"))

    # Drop duplicates based on primary keys
    if "primary_keys" in config:
        cleaned_df = cleaned_df.drop_duplicates(subset=config["primary_keys"])

    return cleaned_df

def add_constraints(session: Session, clean_table: str, config: dict):
    """
    Adds primary and unique key constraints to the cleaned table.

    Args:
        session (Session): Snowflake Snowpark session.
        clean_table (str): Fully qualified name of the cleaned table (schema.table).
        config (dict): Configuration for the table.
    """
    # Add primary key
    if "primary_keys" in config:
        primary_keys = ", ".join(config["primary_keys"])
        session.sql(f"""
            ALTER TABLE {clean_table}
            ADD CONSTRAINT PK_{clean_table.replace('.', '_')} PRIMARY KEY ({primary_keys})
        """).collect()

    # Add unique keys
    if "unique_keys" in config and config["unique_keys"]:
        unique_keys = ", ".join(config["unique_keys"])
        session.sql(f"""
            ALTER TABLE {clean_table}
            ADD CONSTRAINT UK_{clean_table.replace('.', '_')} UNIQUE ({unique_keys})
        """).collect()

    print(f"Constraints added to {clean_table}")


def process_table(session: Session, raw_table: str, config: dict):
    """
    Cleans and processes a raw table based on the provided configuration.

    Args:
        session (Session): Snowflake Snowpark session.
        raw_table (str): Fully qualified name of the raw table (schema.table).
        config (dict): Configuration for the table.
    """
    clean_table_name = config["clean_table"]

    # Step 1: Clean the raw table
    cleaned_df = clean_table(session, raw_table, config)

    # Step 2: Save the cleaned data as a new table
    cleaned_df.write.mode("overwrite").save_as_table(clean_table_name)
    print(f"Cleaned table created: {clean_table_name}")

    # Step 3: Add constraints to the cleaned table
    add_constraints(session, clean_table_name, config)
    print(f"Processing complete for table: {raw_table}")


def process_all_tables(session: Session, table_config: dict):
    """
    Processes all tables based on the configuration and logs the status.

    Args:
        session (Session): Snowflake Snowpark session.
        table_config (dict): Configuration dictionary for all tables.

    Returns:
        dict: Summary of success and failure for each table.
    """
    # Initialize status tracking
    status_summary = {"success": [], "failure": []}

    for raw_table, config in table_config.items():
        try:
            # Process the table
            print(f"Processing table: {raw_table}")
            process_table(session, f"KN_LOGISTICS.SNOW_SQL.{raw_table}", config)

            # Log success
            print(f"Successfully processed table: {raw_table}")
            status_summary["success"].append(raw_table)

        except Exception as e:
            # Log failure and exception details
            print(f"Failed to process table: {raw_table}. Error: {str(e)}")
            status_summary["failure"].append({"table": raw_table, "error": str(e)})

    # Return status summary
    return status_summary


In [ ]:
# Process all tables
status_summary = process_all_tables(session, TABLE_CONFIG)

# Print summary
print("\nProcessing Summary:")
print(f"Successful Tables: {len(status_summary['success'])}")
for table in status_summary["success"]:
    print(f"  - {table}")

print(f"\nFailed Tables: {len(status_summary['failure'])}")
for failure in status_summary["failure"]:
    print(f"  - Table: {failure['table']}, Error: {failure['error']}")


#### 4. Create SNOWSQL View
We will simplify the SNOWSQL schema by joining together the tables and picking only the columns we need. This will be done using Snowpark Dataframe API. Then we'll create a Snowflake stream on that view so that we can incrementally process changes to any of the SNOWSQL tables.

This setup is crucial for ensuring that once the initial data is loaded, we can efficiently manage incremental updates to the SNOWSQL data through the Snowflake stream.

This joining of tables can only be done when the group has finalized cleaning the tables in Snowflake (Snowsight).

In [ ]:
# SNOWFLAKE ADVANTAGE: Snowpark DataFrame API
# SNOWFLAKE ADVANTAGE: Streams for incremental processing (CDC)
# SNOWFLAKE ADVANTAGE: Streams on views

### I NEED HELP ON JOINING THE DATA HERE!!!!!!
# ---------------------------------------------------------------------------------------------

from snowflake.snowpark import Session
import snowflake.snowpark.functions as F
from snowflake.snowpark.functions import col

def create_pos_view(session):
    session.use_schema('SNOWSQL')

    # Define DataFrames for each table with selected column
    application_people = session.table("KN_LOGISTICS.SNOWSQL.APPLICATION_PEOPLE").select(
        F.col("PERSONID"),
        F.col("FULLNAME"),
        F.col("ISEMPLOYEE"),
        F.col("ISSALESPERSON"),
        F.col("PREFERREDNAME"),
        F.col("SEARCHNAME")
    )

    purchasing_purchaseorderlines = session.table("KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES").select(
        F.col("DESCRIPTION"),
        F.col("EXPECTEDUNITPRICEPEROUTER"),
        F.col("ISORDERLINEFINALIZED"),
        F.col("LASTRECEIPTDATE"),
        F.col("ORDERDOUTERS"),
        F.col("PACKAGETYPEID"),
        F.col("PURCHASEORDERID"),
        F.col("PURCHASEORDERLINEID"),
        F.col("RECEIVEDOUTERS"),
        F.col("STOCKITEMID")
    )

    purchasing_purchaseorders = session.table("KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS").select(
        F.col("CONTACTPERSONID"),
        F.col("DELIVERYMETHODID"),
        F.col("EXPECTEDDELIVERYDATE"),
        F.col("ISORDERFINALIZED"),
        F.col("ORDERDATE"),
        F.col("PURCHASEORDERID"),
        F.col("SUPPLIERID"),
        F.col("SUPPLIERREFERENCE")
    )

    purchasing_suppliertransactions = session.table("KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS").select(
        F.col("AMOUNTEXCLUDINGTAX"),
        F.col("FINALIZATIONDATE"),
        F.col("ISFINALIZED"),
        F.col("LASTCOSTPRICE_NUM"),
        F.col("OUTSTANDINGBALANCE"),
        F.col("PAYMENTMETHODID"),
        F.col("PURCHASEORDERID"),
        F.col("SUPPLIERID"),
        F.col("SUPPLIERINVOICENUMBER"),
        F.col("SUPPLIERTRANSACTIONID"),
        F.col("TAXAMOUNT"),
        F.col("TRANSACTIONAMOUNT"),
        F.col("TRANSACTIONDATE"),
        F.col("TRANSACTIONTYPEID")
    )

    sales_customercategories = session.table("KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERCATEGORIES").select(
        F.col("CUSTOMERCATEGORYID"),
        F.col("CUSTOMERCATEGORYNAME")
    )

    
    sales_customers = session.table("KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS").select(
        F.col("CUSTOMERID"),
        F.col("CUSTOMERNAME"),
        F.col("BILLTOCUSTOMERID"),
        F.col("CUSTOMERCATEGORYID"),
        F.col("BUYINGGROUPID"),
        F.col("PRIMARYCONTACTPERSONID"),
        F.col("ALTERNATECONTACTPERSONID"),
        F.col("DELIVERYMETHODID"),
        F.col("DELIVERYCITYID"),
        F.col("CREDITLIMIT"),
        F.col("ACCOUNTOPENEDDATE"),
        F.col("STANDARDDISCOUNTPERCENTAGE"),
        F.col("ISSTATEMENTSENT"),
        F.col("ISONCREDITHOLD"),
        F.col("PAYMENTDAYS"),
        F.col("PHONENUMBER"),
        F.col("WEBSITEURL")
    )

    sales_customertransactions = session.table("KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS").select(
        F.col("AMOUNTEXCLUDINGTAX"),
        F.col("CUSTOMERID"),
        F.col("CUSTOMERTRANSACTIONID"),
        F.col("FINALIZATIONDATE"),
        F.col("INVOICEID"),
        F.col("ISFINALIZED"),
        F.col("OUTSTANDINGBALANCE"),
        F.col("PAYMENTMETHODID"),
        F.col("TAXAMOUNT"),
        F.col("TRANSACTIONAMOUNT"),
        F.col("TRANSACTIONDATE"),
        F.col("TRANSACTIONTYPEID")
    )

    sales_invoicelines = session.table("KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES").select(
        F.col("DESCRIPTION"),
        F.col("EXTENDEDPRICE"),
        F.col("INVOICEID"),
        F.col("INVOICELINEID"),
        F.col("LINEPROFIT"),
        F.col("PACKAGETYPEID"),
        F.col("QUANTTY"),
        F.col("STOCKITEMID"),
        F.col("TAXAMOUNT"),
        F.col("TAXRATE"),
        F.col("UNITPRICE")
    )

    sales_invoices = session.table("KN_LOGISTICS.SNOWSQL.SALES_INVOICES").select(
        F.col("ACCOUNTSPERSONID"),
        F.col("BILLTOCUSTOMERID"),
        F.col("CONFIRMEDDELIVERYTIME"),
        F.col("CONFIRMEDRECEIVEDBY"),
        F.col("CONTACTPERSONID"),
        F.col("CUSTOMERID"),
        F.col("CUSTOMERPURCHASEORDERNUMBER"),
        F.col("DELIVERYMETHODID"),
        F.col("INVOICEDATE"),
        F.col("INVOICEID"),
        F.col("ORDERID"),
        F.col("PACKEDBYPERSONID"),
        F.col("SALESPERSONPERSONID"),
        F.col("TOTALCHILLERITEMS"),
        F.col("TOTALDRYITEMS")
    )

    sales_orderlines = session.table("KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES").select(
        F.col("DESCRIPTION"),
        F.col("ORDERID"),
        F.col("ORDERLINEID"),
        F.col("PACKAGETYPEID"),
        F.col("PICKEDQUANTITY"),
        F.col("PICKINGCOMPLETEDWHEN"),
        F.col("QUANTITY"),
        F.col("STOCKITEMID"),
        F.col("TAXRATE"),
        F.col("UNITPRICE")
    )

    sales_ordes = session.table("KN_LOGISTICS.SNOWSQL.SALES_ORDERS").select(
        F.col("CONTACTPERSONID"),
        F.col("CUSTOMERID"),
        F.col("CUSTOMERPURCHASEORDERNUMBER"),
        F.col("EXPECTEDDELIVERYDATE"),
        F.col("ISUNDERSUPPLYBACKORDERED"),
        F.col("ORDERDATE"),
        F.col("ORDERID"),
        F.col("PICKEDBYPERSONID"),
        F.col("PICKINGCOMPLETEDWHEN"),
        F.col("SALESPERSONPERSONID")
    )

    warehouse_coldroomtemperatures = session.table("KN_LOGISTICS.SNOWSQL.WAREHOUSE_COLDROOMTEMPERATURES").select(
        F.col("COLDROOMSENSORNUMBER"),
        F.col("COLDROOMTEMPERATUREID"),
        F.col("RECORDEDWHEN"),
        F.col("RECORDEDWHEN_T"),
        F.col("RECORDEDWHEN_TS"),
        F.col("TEMPERATURE"),
        F.col("TEMPERATURE_FLOAT")
    )

    warehouse_stockitemholdings = session.table("KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMHOLDINGS").select(
        F.col("BINLOCATION"),
        F.col("LASTCOSTPRICE"),
        F.col("LASTSTOCKTAKEQUANTITY"),
        F.col("QUANTITYONHAND"),
        F.col("REORDERLEVEL"),
        F.col("STOCKITEMID"),
        F.col("TARGETSTOCKLEVEL")
    )
    
    warehouse_stockitems = session.table("KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS").select(
        F.col("STOCKITEMID"),
        F.col("STOCKITEMNAME"),
        F.col("UNITPACKAGEID"),
        F.col("OUTERPACKAGEID"),
        F.col("BRAND"),
        F.col("SIZE"),
        F.col("LEADTIMEDAYS"),
        F.col("QUANTITYPEROUTER"),
        F.col("ISCHILLERSTOCK"),
        F.col("TAXRATE"),
        F.col("UNITPRICE"),
        F.col("RECOMMENDEDRETAILPRICE"),
        F.col("TYPICALWEIGHTPERUNIT"),
        F.col("SUPPLIERID"),
        F.col("COLORID")
    )

    warehouse_stockitemtransactions = session.table("KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS").select(
        F.col("CUSTOMERID"),
        F.col("INVOICEID"),
        F.col("PURCHASEORDERID"),
        F.col("QUANTITY"),
        F.col("STOCKITEMID"),
        F.col("STOCKITEMTRANSACTIONID"),
        F.col("SUPPLIERID"),
        F.col("TRANSACTIONOCCURREDWHEN"),
        F.col("TRANSACTIONTYPEID")
    )

    warehouse_vehicletemperatures = session.table("KN_LOGISTICS.SNOWSQL.WAREHOUSE_VEHICLETEMPERATURES").select(
        F.col("CHILLERSENSORNUMBER"),
        F.col("FULLSENSORDATA"),
        F.col("RECORDEDWHEN"),
        F.col("TEMPERATURE"),
        F.col("VEHICLEREGISTRATION"),
        F.col("VEHICLETEMPERATUREID")
    )



# --------------------------------------------------------------------------------------------
# --------------------------------------------- TO NEHAA ----------------------------------------------------
# Tables in this notebook
# Sales.CustomerTransactions
# Sales.CustomerCategories
# Application.People
# Purchasing.SupplierTransactions
# Purchasing.PurchaseOrderLines
# Warehouse.ColdRoomTemperatures
# Warehouse.VehicleTemperatures
# Sales.Orders
# Purchasing.PurchaseOrders
# Warehouse.StockItemTransactions
# Warehouse.StockItems
# Warehouse.StockItemHoldings
# Sales.Customers
# Sales.Invoices
# Sales.InvoiceLines
# Sales.Orderlines


# TO NEHAA:
    # DO THE JOINING HERE
    # U NEED TO EDIT THE TABLES THAT DONT EXIST

    # Join Purchasing Tables
    purchase_orders_with_details = purchasing_purchaseorders.join(
        purchasing_suppliers,
        purchasing_purchaseorders["SUPPLIERID"] == purchasing_suppliers["SUPPLIERID"]
    ).join(
        application_people,
        purchasing_purchaseorders["CONTACTPERSONID"] == application_people["PERSONID"]
    ).join(
        application_deliverymethods,
        purchasing_purchaseorders["DELIVERYMETHODID"] == application_deliverymethods["DELIVERYMETHODID"]
    ).select(
        purchasing_purchaseorders["PURCHASEORDERID"],
        purchasing_purchaseorders["ORDERDATE"],
        application_people["FULLNAME"].alias("CONTACT_PERSON"),
        purchasing_suppliers["SUPPLIERNAME"],
        application_deliverymethods["DELIVERYMETHODNAME"]
    )

    # Join Sales Tables
    sales_data = sales_invoices.join(
        sales_customers,
        sales_invoices["CUSTOMERID"] == sales_customers["CUSTOMERID"]
    ).join(
        sales_orderlines,
        sales_invoices["ORDERID"] == sales_orderlines["ORDERID"]
    ).join(
        application_people,
        sales_invoices["CONTACTPERSONID"] == application_people["PERSONID"]
    ).select(
        sales_invoices["INVOICEID"],
        sales_invoices["INVOICEDATE"],
        sales_customers["CUSTOMERNAME"],
        application_people["FULLNAME"].alias("CONTACT_PERSON"),
        sales_orderlines["DESCRIPTION"],
        sales_orderlines["QUANTITY"],
        sales_orderlines["UNITPRICE"]
    )

    # Join Stock Tables
    stock_with_transactions = warehouse_stockitems.join(
        warehouse_stockitemtransactions,
        warehouse_stockitems["STOCKITEMID"] == warehouse_stockitemtransactions["STOCKITEMID"]
    ).join(
        warehouse_colors,
        warehouse_stockitems["COLORID"] == warehouse_colors["COLORID"]
    ).select(
        warehouse_stockitems["STOCKITEMNAME"],
        warehouse_colors["COLORNAME"],
        warehouse_stockitemtransactions["TRANSACTIONOCCURREDWHEN"],
        warehouse_stockitemtransactions["QUANTITY"]
    )

    # Final Unified DataFrame
    final_dataframe = sales_data.join(
        stock_with_transactions,
        sales_data["DESCRIPTION"] == stock_with_transactions["STOCKITEMNAME"],
        how="left"
    #).join(
    #    cities_with_countries,
     #   sales_customers["DELIVERYCITYID"] == cities_with_countries["CITYID"],
      #  how="left"
    ).join(
        purchase_orders_with_details,
        sales_data["INVOICEID"] == purchase_orders_with_details["PURCHASEORDERID"],
        how="left"
    ).select(
        sales_data["INVOICEID"],
        sales_data["INVOICEDATE"],
        sales_data["CUSTOMERNAME"],
        sales_data["CONTACT_PERSON"],
        sales_data["DESCRIPTION"],
        sales_data["QUANTITY"],
        stock_with_transactions["COLORNAME"],
        #cities_with_countries["CITYNAME"].alias("DELIVERY_CITY"),
        #cities_with_countries["COUNTRYNAME"].alias("DELIVERY_COUNTRY"),
        purchase_orders_with_details["SUPPLIERNAME"],
        purchase_orders_with_details["DELIVERYMETHODNAME"]
    )

    final_dataframe.create_or_replace_view("SQL_FLATTENED_V")
    

def create_pos_view_stream(session):
    session.use_schema('SNOWSQL')
    _ = session.sql('CREATE OR REPLACE STREAM SNOWSQL_FLATTENED_V_STREAM \
                        ON VIEW SQL_FLATTENED_V \
                        SHOW_INITIAL_ROWS = TRUE').collect()

def test_pos_view(session):
    session.use_schema('SNOWSQL')
    tv = session.table('SQL_FLATTENED_V')
    tv.limit(5).show()


In [ ]:
DROP VIEW IF EXISTS DetailedSalesTransactions;
DROP VIEW IF EXISTS PurchasingInsights;
DROP VIEW IF EXISTS WarehouseOverview;
DROP VIEW IF EXISTS OrderHandling;



-- HYBRID VIEW 1: DETAILED SALES TRANSACTIONS
CREATE OR ALTER VIEW DetailedSalesTransactions AS
SELECT 
    ct.CUSTOMERTRANSACTIONID,
    ct.TRANSACTIONDATE,
    ct.AMOUNTEXCLUDINGTAX,
    ct.TAXAMOUNT,
    ct.TRANSACTIONAMOUNT,
    ct.OUTSTANDINGBALANCE,
    c.CUSTOMERNAME,
    cc.CUSTOMERCATEGORYNAME
FROM 
    SALES_CUSTOMERTRANSACTIONS ct
JOIN 
    SALES_CUSTOMERS c ON ct.CUSTOMERID = c.CUSTOMERID
JOIN 
    SALES_CUSTOMERCATEGORIES cc ON c.CUSTOMERCATEGORYID = cc.CUSTOMERCATEGORYID;

-- HYBRID VIEW 2: PURCHASING INSIGHTS
CREATE OR ALTER VIEW PurchasingInsights AS
SELECT 
    po.PURCHASEORDERID,
    po.ORDERDATE,
    po.DELIVERYMETHODID,
    pol.STOCKITEMID
FROM 
    PURCHASING_PURCHASEORDERS po
JOIN 
    PURCHASING_PURCHASEORDERLINES pol ON po.PURCHASEORDERID = pol.PURCHASEORDERID
JOIN 
    PURCHASING_SUPPLIERTRANSACTIONS st ON po.SUPPLIERID = st.SUPPLIERID;

-- HYBRID VIEW 3: WAREHOUSE OVERVIEW
CREATE OR ALTER VIEW WarehouseOverview AS
SELECT 
    si.STOCKITEMID,
    si.STOCKITEMNAME,
    si.COLORID,
    si.SIZE,
    si.LEADTIMEDAYS,
    si.QUANTITYPEROUTER,
    sh.QUANTITYONHAND,
    sit.TRANSACTIONOCCURREDWHEN,
    sit.QUANTITY AS TRANSACTIONQUANTITY
FROM 
    WAREHOUSE_STOCKITEMS si
LEFT JOIN 
    WAREHOUSE_STOCKITEMHOLDINGS sh ON si.STOCKITEMID = sh.STOCKITEMID
LEFT JOIN 
    WAREHOUSE_STOCKITEMTRANSACTIONS sit ON si.STOCKITEMID = sit.STOCKITEMID;

-- HYBRID VIEW 4: ORDER HANDLING
CREATE OR ALTER VIEW OrderHandling AS
SELECT 
    o.ORDERID,
    o.ORDERDATE,
    o.EXPECTEDDELIVERYDATE,
    o.CUSTOMERID,
    o.SALESPERSONPERSONID,
    p.FULLNAME AS SALESPERSONNAME
FROM 
    SALES_ORDERS o
JOIN 
    APPLICATION_PEOPLE p ON o.SALESPERSONPERSONID = p.PERSONID;


In [ ]:
SELECT * FROM DetailedSalesTransactions;


In [ ]:
SELECT * FROM PurchasingInsights;


In [ ]:
SELECT * FROM WarehouseOverview;


In [ ]:
SELECT * FROM OrderHandling;


In [ ]:
# use this place to validate the tables
# test the code
create_pos_view(session)

In [ ]:
# visualize
# Test the view created by create_pos_view
def validate_view(session):
    session.use_schema('SNOWSQL')
    # Load the view into a Snowpark DataFrame
    df = session.table("SQL_FLATTENED_V")
    # Display the first few rows for validation
    df.show()

validate_view(session)

## Data UDF

During this step we will be creating and deploying our first Snowpark Python object to Snowflake, a user-defined function (or UDF). To begin with the UDF will be very basic, but in a future step we'll update it to include a third-party Python package. Also in this step you will be introduced to the new SnowCLI, a new developer command line tool. SnowCLI makes building and deploying Snowpark Python objects to Snowflake a consistent experience for the developer. More details below on SnowCLI.

## Orders Update Sproc

During this step we will be creating and deploying our first Snowpark Python stored procedure (or sproc) to Snowflake. This sproc will merge changes from the `HARMONIZED.POS_FLATTENED_V_STREAM` stream into our target `HARMONIZED.ORDERS` table.

### Running the Sproc Locally
To test the procedure locally, you will execute the following script.

In [ ]:
# close session
session.close()